In [14]:
import cv2
import numpy as np

In [15]:
img_dir = '/Users/crossrunway/Downloads/gt3.jpeg'
img = cv2.imread(img_dir)

In [16]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [17]:
g_code = ""
for contour in contours:
    for point in contour:
        x, y = point[0]
        g_code += "G01 X{} Y{}\n".format(x, y)

In [18]:
print(g_code)

G01 X0 Y0
G01 X0 Y167
G01 X299 Y167
G01 X299 Y0



In [19]:
# Read in image
img = cv2.imread(img_dir, 0)
img = 255 - img
img = cv2.resize(img, (6000,6000))
h,w = img.shape
cutter_size = 20
buffer = int(cutter_size * 1.25)
img2 = np.zeros((h+buffer*2, w+buffer*2), np.uint8)
img2[buffer:buffer+h, buffer:buffer+w] = img
img = img2.copy()

h,w = img.shape

In [20]:
# Define parameters
pixels_to_inch = 1000
depth_per_cut = 125 # Value in thousands
depth = 500 # Value in thousands
jog_speed = "15"
feed_speed = "9"
plunge_speed = "3"
current_depth = 0
cuts = 2

In [21]:
# Find the center of the part - there should only be one contour
contours, _ = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contour = contours[1]

kernel = np.ones((20,20), np.uint8)
dilate = cv2.dilate(img, kernel, iterations = 4)

contours_dilate = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour_dilate = contours_dilate[0]

In [22]:
# Change the contours from arrays to lists
outter = []
for point in contour_dilate[0]:
    outter.append(tuple(point[0]))
inner = []
for point in contour:
    inner.append(tuple(point[0]))

h,w = img.shape
bg = np.zeros((h,w,3), np.uint8)
bg = img.copy()
bg = cv2.drawContours(bg, [contour], -1, (255,255,0), 2)
bg = cv2.drawContours(bg, contour_dilate, -1, (0,0,255), 2)
cv2.imshow('img', cv2.resize(bg, (987,987)))
cv2.waitKey(0)
cv2.destroyAllWindows()

bg = np.zeros((h,w,3), np.uint8)
for i in range(len(inner)-1):
    a = inner[i]
    b = inner[i-1]
    cv2.line(bg, a, b, (0,255,0), cutter_size)
for i in range(len(outter)-1):
    a = outter[i]
    b = outter[i+1]
    cv2.line(bg, a, b, (255,0,255), cutter_size)
    
bg = cv2.drawContours(bg, [contour], -1, (255,255,0), 2)
bg = cv2.drawContours(bg, contour_dilate, -1, (0,0,255), 2)
cv2.imshow('img', cv2.resize(bg, (987,987)))
cv2.waitKey()
cv2.destroyAllWindows()

In [23]:
# Create string for g_code
g_code = "%" + "\n"
g_code += "G64 P0.001" + "\n"
g_code += "M6 T1" + "\n"
g_code += "M3 S5000" + "\n"
g_code += "G1 F" + jog_speed + "\n"
g_code += "G1X0.0000Y0.0000Z0.1000" + "\n"
g_code += "G1 F" + jog_speed + "\n"

In [24]:
# Jog to the first point in the contour
dilate_origin = tuple(contour_dilate[0][0][0])
g_code += "G1 X" + str(dilate_origin[0]/1000.00) + "Y" + str(dilate_origin[1]/1000.00) + "\n"

origin = tuple(contour[0][0])

previous_point = dilate_origin

In [25]:
# Change the contours from arrays to lists
outter = []
for point in contour_dilate[0]:
    outter.append(tuple(point[0]))
inner = []
for point in contour:
    inner.append(tuple(point[0]))

In [26]:
# Make cuts
for cut in range(cuts):
    # Plunge into the material
    g_code += "G1 F" + plunge_speed + "\n"
    current_depth -= depth_per_cut
    g_code += "G1 Z" + str(current_depth/1000.00) + "\n"

    # Cut the bigger contour
    g_code += "G1 F" + feed_speed + "\n"

    # Iterate through each point in the dilated contour
    for point in outter:
        x = point[0]/1000.00
        y = point[1]/1000.00
        g_code += "X" + str(x) + "Y" + str(y) + "\n"

      
    # Go back to the origin
    g_code += "G1 X" + str(dilate_origin[0]/1000.00) + "Y" + str(dilate_origin[1]/1000.00) + "\n"
        
    # Iterate through each point in the contour
    for point in inner:
        x = point[0]/1000.00
        y = point[1]/1000.00
        g_code += "X" + str(x) + "Y" + str(y) + "\n"
        
    # Go back to the origin
    g_code += "G1 X" + str(origin[0]/1000.00) + "Y" + str(origin[1]/1000.00) + "\n"
g_code += "G0Z.1"
g_code += "M5\n"
g_code += "M02\n"
g_code += "%"

In [27]:
print(g_code)

%
G64 P0.001
M6 T1
M3 S5000
G1 F15
G1X0.0000Y0.0000Z0.1000
G1 F15
G1 X1.812Y1.974
G1 F3
G1 Z-0.125
G1 F9
X1.812Y1.974
X1.811Y1.975
X1.8Y1.975
X1.799Y1.976
X1.792Y1.976
X1.791Y1.977
X1.787Y1.977
X1.786Y1.978
X1.783Y1.978
X1.782Y1.979
X1.78Y1.979
X1.779Y1.98
X1.778Y1.98
X1.777Y1.981
X1.776Y1.981
X1.774Y1.983
X1.773Y1.983
X1.771Y1.985
X1.771Y1.986
X1.769Y1.988
X1.769Y1.989
X1.768Y1.99
X1.768Y1.991
X1.767Y1.992
X1.767Y1.994
X1.766Y1.995
X1.766Y1.998
X1.765Y1.999
X1.765Y2.003
X1.764Y2.004
X1.764Y2.085
X1.762Y2.087
X1.758Y2.087
X1.757Y2.088
X1.756Y2.088
X1.749Y2.095
X1.749Y2.096
X1.748Y2.097
X1.748Y2.098
X1.747Y2.099
X1.747Y2.101
X1.746Y2.102
X1.746Y2.105
X1.745Y2.106
X1.745Y2.11
X1.744Y2.111
X1.744Y2.118
X1.742Y2.12
X1.742Y2.131
X1.741Y2.132
X1.741Y2.143
X1.74Y2.144
X1.74Y2.261
X1.739Y2.262
X1.736Y2.262
X1.735Y2.261
X1.726Y2.261
X1.725Y2.26
X1.708Y2.26
X1.707Y2.259
X1.554Y2.259
X1.553Y2.26
X1.517Y2.26
X1.516Y2.261
X1.498Y2.261
X1.497Y2.262
X1.491Y2.262
X1.49Y2.263
X1.486Y2.263
X1.485Y2.264
